In [1]:
try:
    import scapy.all as scapy
except ImportError:
    import scapy

try:
    # This import works from the project directory
    import scapy_http.http
except ImportError:
    # If you installed this package via pip, you just need to execute this
    from scapy.layers import http

read the pcap file

In [10]:
def get_http_payload(ether):
    return ether.payload.payload.payload.payload

def count_char(pcapFile):
    char_set = set()
    with scapy.PcapReader(pcapFile) as pcap_reader:
        for pkt in pcap_reader:
            http_payload = get_http_payload(pkt)
            http_payload.remove_payload(
            )
            tmp = str(bytes(http_payload), encoding='utf-8')
            for c in tmp:
                char_set.add(c)
    print(char_set)
    print('共有字符个数：', len(char_set))
    return(char_set)

In [11]:
pcapFile_weibo = "../data/weibo_http_request.pcap"
pcapFile_zhihu = "../data/zhihu/zhihu_req.pcap"
# 统计微博的字符
weibo_set = count_char(pcapFile_weibo)

{'.', 'Y', 'D', 'k', '/', 'n', 'A', 'o', '1', 'Q', '8', '\r', 'Z', 'x', ')', 'g', 'B', 'q', 't', 'X', 'I', '&', '~', 'K', '0', 's', 'T', 'S', 'p', 'a', 'C', ' ', '=', 'y', 'v', 'd', 'V', 'c', '9', 'f', '}', '?', 'i', '(', 'J', '2', '5', 'u', 'e', '-', 'l', '*', '"', '{', 'w', 'R', 'j', '4', '_', 'F', 'H', 'r', '6', 'N', '3', 'E', '+', '\n', 'L', ';', 'z', ':', '7', ',', 'O', '%', 'b', 'U', 'W', 'M', 'm', 'P', 'G', 'h'}
共有字符个数： 84


In [12]:
# 统计知乎的字符
zhihu_set = count_char(pcapFile_zhihu)

{'.', 'Y', 'D', 'k', '/', 'n', 'A', 'o', '1', 'Q', '8', '\r', 'Z', 'x', ')', 'g', 'B', 'q', 't', 'X', 'I', '&', 'K', '0', 's', 'T', 'S', 'p', 'a', 'C', ' ', '=', 'v', 'y', 'd', 'V', 'c', '9', 'f', '?', 'i', '(', 'J', '2', '5', 'e', 'u', '-', 'l', 'w', 'R', 'j', '4', '_', 'F', 'H', 'r', '6', 'E', 'N', '3', '+', '\n', 'L', '7', 'z', ':', ';', ',', 'O', '%', 'b', 'U', 'W', 'M', 'm', 'P', 'G', 'h'}
共有字符个数： 79


In [13]:
for c in weibo_set:
    if c not in zhihu_set:
        print(c)

~
}
*
"
{


In [14]:
for c in zhihu_set:
    if c not in weibo_set:
        print(c)

In [ ]:
# one hot encode
char2int = dict((c, i) for (c, i) in enumerate(weibo_set))
int2char = dict((i, c) for (c, i) in enumerate(weibo_set))
def onehot_encode(data)
    integer_encoded = [char2int[char] for char in data] # 获得对data编码的list   
    onehot_encoded = list()
    for value in integer_encoded:
           letter = [0 for _ in range(len(weibo_set))]   # 生成一个全0的list
           letter[value] = 1
           onehot_encoded.append(letter)
    print(onehot_encoded)   # 对data的编码结果
    # invert encoding (从编码获取原字符)
    inverted = int_to_char[argmax(onehot_encoded[0])]
    print(inverted)
    return onehot_encoded